In [5]:
import pandas as pd
import numpy as np
import os
import glob
from querybuilder_client import QuerybuilderClient
from queryrunner_client import Client
import datetime
import base64
from IPython.display import HTML
import warnings
import pygsheets
import ast
import IPython
from numpy import nan
import json
# import pypostmaster
warnings.filterwarnings('ignore')

from dateutil.parser import parse
from time import sleep
from datetime import datetime
from datetime import timedelta

import os
from oauth2client.service_account import ServiceAccountCredentials

qr = Client(user_email='mrajor@ext.uber.com', consumer_name='marketplace-mobility-ops-analytics')

gc = pygsheets.authorize(service_file='meehir.json')

wsaccess = pygsheets.authorize(service_file='meehir.json')
main_sheet = wsaccess.open_by_key('1lBKqpEENuGZAPXsLgEr9OWoqe8MniU4bjolehpwBwl8')
details = main_sheet.worksheet_by_title('manually_handled_npi')
qr = Client(user_email='mrajor@ext.uber.com', consumer_name='marketplace-mobility-ops-analytics')

12/23/2024 09:22:13 AM  Obtained utoken email from file 
12/23/2024 09:22:14 AM  Obtained utoken from utoken cli 
12/23/2024 09:22:14 AM  Obtained utoken email from file 
12/23/2024 09:22:15 AM  Obtained utoken from utoken cli 


In [6]:
sheet=details.get_as_df()
sheet

,Report,tab_name,sheet
0,zgirSXWh5,Uber_raw_data,https://docs.google.com/spreadsheets/d/1lBKqpE...
1,qFbg9Uofx,ola_raw_data,https://docs.google.com/spreadsheets/d/1lBKqpE...


In [7]:
for i in range(0,2):
    print(sheet.iloc[i,2])

https://docs.google.com/spreadsheets/d/1lBKqpEENuGZAPXsLgEr9OWoqe8MniU4bjolehpwBwl8/edit?gid=1719304691#gid=1719304691
https://docs.google.com/spreadsheets/d/1lBKqpEENuGZAPXsLgEr9OWoqe8MniU4bjolehpwBwl8/edit?gid=0#gid=0


In [8]:
today=datetime.today()
start_date=today-timedelta(days=1)
end_date=today-timedelta(days=0)
start=start_date.strftime("%Y-%m-%d")
end=end_date.strftime("%Y-%m-%d")
print(start)
print(end)

2024-12-22
2024-12-23


In [9]:
week_number=start_date.isocalendar()[1]
print(week_number)

51


In [6]:
def upload_in_wks_replace(dash_link,tab_name,df):
    wks=gc.open_by_url(dash_link).worksheet_by_title(tab_name)
    wks.clear()
    lastrow=len(wks.get_col(1,include_tailing_empty=False)) + 1
    wks.set_dataframe(df,start='A{}'.format(lastrow),copy_head=False)
    print("data uploaded")

In [7]:
def upload_in_wks(dash_link,tab_name,df):
    wks=gc.open_by_url(dash_link).worksheet_by_title(tab_name)
    wks.add_rows(df.shape[0])
    lastrow=len(wks.get_col(1,include_tailing_empty=False)) + 1
    wks.set_dataframe(df,start='A{}'.format(lastrow),copy_head=False)
    print("data uploaded")

In [8]:
def make_data_types_common(df):
    df['day'] = df['day'].astype(str)
    df['Time'] = df['Time'].astype(str)
    df['city_name'] = df['city_name'].astype(str)
    df['scr_geo'] = df['scr_geo'].astype(str)
    df['dest_geo'] = df['dest_geo'].astype(str)
    df['Distance_bucket'] = df['Distance_bucket'].astype(str)
#     df['week'] = df['week'].astype(str)
    return df

In [9]:
def merging_df(uber_df, df):
    print("Uber DataFrame Columns:", uber_df.columns.tolist())
    print("Input DataFrame Columns:", df.columns.tolist())
    
    # Ensure 'day' is in datetime format
    df['day'] = pd.to_datetime(df['day'])
    df['day'] = df['day'].dt.strftime('%Y-%m-%d')

    # Ensure numeric types for relevant columns, check if they exist
    numeric_columns = ['sum_fare', 'sum_dist', 'points', 'requests', 'req weighted dist']
    for col in numeric_columns:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        if col in uber_df.columns:
            uber_df[col] = pd.to_numeric(uber_df[col], errors='coerce')

    # Merge DataFrames
    merged_df = pd.merge(uber_df, df, on=['day', 'Time', 'city_name', 'scr_geo', 'dest_geo', 'Distance_bucket'], how='inner')

    # Group and transform
    if 'sum_fare' in merged_df.columns:
        merged_df['sum_fare'] = merged_df.groupby(['day', 'Time', 'city_name', 'scr_geo', 'dest_geo', 'Distance_bucket'])['sum_fare'].transform('sum')
    if 'sum_dist' in merged_df.columns:
        merged_df['sum_dist'] = merged_df.groupby(['day', 'Time', 'city_name', 'scr_geo', 'dest_geo', 'Distance_bucket'])['sum_dist'].transform('sum')
    if 'points' in merged_df.columns:
        merged_df['points'] = merged_df.groupby(['day', 'Time', 'city_name', 'scr_geo', 'dest_geo', 'Distance_bucket'])['points'].transform('sum')

    merged_df['day'] = pd.to_datetime(merged_df['day'])
    merged_df['week'] = merged_df['day'].dt.isocalendar().week

    if 'requests' in merged_df.columns:
        merged_df['requests'] = merged_df.groupby(['day', 'Time', 'city_name', 'scr_geo', 'dest_geo', 'Distance_bucket'])['requests'].transform('sum')
    if 'req weighted dist' in merged_df.columns:
        merged_df['uber_dist'] = merged_df.groupby(['day', 'Time', 'city_name', 'scr_geo', 'dest_geo', 'Distance_bucket'])['req weighted dist'].transform('sum')

    # Convert requests to int
    if 'requests' in merged_df.columns:
        merged_df['requests'] = merged_df['requests'].astype(int)

    # Calculate weighted fare only if columns exist
    if 'sum_fare' in merged_df.columns and 'sum_dist' in merged_df.columns and 'uber_dist' in merged_df.columns:
        merged_df['req weighted fare'] = (merged_df['sum_fare'] / merged_df['sum_dist']) * merged_df['uber_dist']
    if 'requests' in merged_df.columns and 'sum_dist' in merged_df.columns:
        merged_df['req weighted dist'] = merged_df['requests'] * merged_df['sum_dist']

    return merged_df


In [10]:
def merging_df_rapido(uber_df, df):
    print("Rapido Uber DataFrame Columns:", uber_df.columns.tolist())
    print("Input DataFrame Columns:", df.columns.tolist())
    
    df['day'] = pd.to_datetime(df['day'])
    df['day'] = df['day'].dt.strftime('%Y-%m-%d')

    # Ensure numeric types for relevant columns, check if they exist
    numeric_columns = ['sum_fare_pre', 'sum_fare_post', 'sum_dist', 'points', 'requests', 'req weighted dist']
    for col in numeric_columns:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        if col in uber_df.columns:
            uber_df[col] = pd.to_numeric(uber_df[col], errors='coerce')

    merged_df = pd.merge(uber_df, df, on=['day', 'Time', 'city_name', 'scr_geo', 'dest_geo', 'Distance_bucket'], how='inner')

    if 'sum_fare_pre' in merged_df.columns:
        merged_df['sum_fare_pre'] = merged_df.groupby(['day', 'Time', 'city_name', 'scr_geo', 'dest_geo', 'Distance_bucket'])['sum_fare_pre'].transform('sum')
    if 'sum_fare_post' in merged_df.columns:
        merged_df['sum_fare_post'] = merged_df.groupby(['day', 'Time', 'city_name', 'scr_geo', 'dest_geo', 'Distance_bucket'])['sum_fare_post'].transform('sum')
    if 'sum_dist' in merged_df.columns:
        merged_df['sum_dist'] = merged_df.groupby(['day', 'Time', 'city_name', 'scr_geo', 'dest_geo', 'Distance_bucket'])['sum_dist'].transform('sum')
    if 'points' in merged_df.columns:
        merged_df['points'] = merged_df.groupby(['day', 'Time', 'city_name', 'scr_geo', 'dest_geo', 'Distance_bucket'])['points'].transform('sum')

    #merged_df['week'] = merged_df['day'].dt.isocalendar().week
    merged_df['week'] = week_number

    if 'requests' in merged_df.columns:
        merged_df['requests'] = merged_df.groupby(['day', 'Time', 'city_name', 'scr_geo', 'dest_geo', 'Distance_bucket'])['requests'].transform('sum')
    if 'req weighted dist' in merged_df.columns:
        merged_df['uber_dist'] = merged_df.groupby(['day', 'Time', 'city_name', 'scr_geo', 'dest_geo', 'Distance_bucket'])['req weighted dist'].transform('sum')

    if 'requests' in merged_df.columns:
        merged_df['requests'] = merged_df['requests'].astype(int)

    if 'sum_fare_pre' in merged_df.columns and 'sum_dist' in merged_df.columns and 'uber_dist' in merged_df.columns:
        merged_df['req weighted fare_pre'] = (merged_df['sum_fare_pre'] / merged_df['sum_dist']) * merged_df['uber_dist']
    if 'sum_fare_post' in merged_df.columns and 'sum_dist' in merged_df.columns and 'uber_dist' in merged_df.columns:
        merged_df['req weighted fare_post'] = (merged_df['sum_fare_post'] / merged_df['sum_dist']) * merged_df['uber_dist']
    
    if 'requests' in merged_df.columns and 'sum_dist' in merged_df.columns:
        merged_df['req weighted dist'] = merged_df['requests'] * merged_df['sum_dist']

    return merged_df


In [11]:
# Main Logic
ola_uber_df = pd.DataFrame()
app_df = pd.DataFrame()
rapido_uber_df = pd.DataFrame()
moto_uber_df_web = pd.DataFrame()
moto_uber_df_app = pd.DataFrame()

for i in range(0, 2):
    df = qr.execute_report(sheet.iloc[i, 0], parameters={"start": start, "end": end}, datacenter='phx2').to_pandas()
    
    if sheet.iloc[i, 1] == 'Uber_raw_data':
        df['day'] = pd.to_datetime(df['day'])
        df['week'] = df['day'].dt.isocalendar().week
        df['req weighted fare'] = df['sum_fare']
        df['req weighted dist'] = df['sum_dist']
        ola_uber_df = df
        rapido_uber_df = df
        app_df = df
        upload_in_wks(sheet.iloc[i, 2], sheet.iloc[i, 1], df) 
        
    elif sheet.iloc[i, 1] == 'ola_raw_data':
        print(app_df)
        app_df.drop(columns=['sum_fare', 'sum_dist', 'points', 'req weighted fare', 'week'], inplace=True)
        app_df = make_data_types_common(app_df)
        df = make_data_types_common(df)
        new_df = merging_df(app_df, df)
        temp_df = new_df[['day', 'Time', 'city_name', 'scr_geo', 'dest_geo', 'Distance_bucket',
                           'sum_fare', 'sum_dist', 'points', 'week', 'requests',
                           'req weighted fare', 'uber_dist', 'req weighted dist']]
        upload_in_wks(sheet.iloc[i, 2], sheet.iloc[i, 1], temp_df)  


12/22/2024 07:00:33 AM  Could not obtain utoken from cli: b'/bin/sh: 1: usso: not found\n'
Trying to obtain from file 
12/22/2024 07:00:33 AM  Obtained utoken for user 
12/22/2024 07:00:33 AM  Fetching metadata for Report zgirSXWh5 
12/22/2024 07:00:33 AM  Loaded object metadata. 
12/22/2024 07:00:33 AM  Templating query for report zgirSXWh5 
12/22/2024 07:00:33 AM  Templated query successfully. 
12/22/2024 07:00:33 AM Send tier_metadata {'report_id': 'zgirSXWh5'} to Queryrunner V2.
12/22/2024 07:00:33 AM  [Polling] c007cf47-4cf1-41c9-9097-43be1d65adeb 
12/22/2024 07:00:33 AM  [Status] created 
12/22/2024 07:00:34 AM  [Status] started validation 
12/22/2024 07:00:38 AM  [Status] started auth check 
12/22/2024 07:00:42 AM  [Status] started waiting to execute 
12/22/2024 07:02:18 AM  [Status] started execution 
12/22/2024 07:04:44 AM  UToken has expired. Generating new utoken. 
12/22/2024 07:04:44 AM  Obtained utoken email from file 
12/22/2024 07:04:44 AM  Obtained utoken from utoken cl

data uploaded


12/22/2024 07:08:47 AM  [Polling] 472e87d7-3f8e-40c1-bce8-74983e2d09e4 
12/22/2024 07:08:47 AM  [Status] created 
12/22/2024 07:08:48 AM  [Status] started validation 
12/22/2024 07:08:49 AM  [Status] started auth check 
12/22/2024 07:08:50 AM  [Status] started waiting to execute 
12/22/2024 07:08:51 AM  [Status] started execution 
12/22/2024 07:09:07 AM  [Status] completed success 
12/22/2024 07:09:07 AM  [Query Success] completed success 


            day                  Time   city_name        scr_geo  \
0    2024-12-20    Evening_peak_17_20      Mumbai           city   
1    2024-12-20  Afternoon_hour_11_16      Mumbai           city   
2    2024-12-20    Evening_peak_17_20   Delhi NCR  Uttar Pradesh   
3    2024-12-20     Morning_peak_7_10   Delhi NCR          Delhi   
4    2024-12-20     Morning_peak_7_10      Mumbai           city   
...         ...                   ...         ...            ...   
4902 2024-12-20           Night_21_23     Lucknow        airport   
4903 2024-12-20        Graveyard_0_06   Ahmedabad           city   
4904 2024-12-20    Evening_peak_17_20  Chandigarh           city   
4905 2024-12-20        Graveyard_0_06   Delhi NCR          Delhi   
4906 2024-12-20        Graveyard_0_06       Kochi        airport   

           dest_geo Distance_bucket      sum_fare      sum_dist points  \
0              city          50 km+  4.409719e+07  1.999686e+06  25015   
1           airport        16-18 km

IndexError: index 2 is out of bounds for axis 0 with size 2